# SQLite3
## database interaction from Python


To interact with a databse sqlite3 and pandas are excelent starting points.

In [1]:
# import
import sqlite3
import pandas as pd

As an example database we will work with a dataset downloaded from [BOLD database](https://boldsystems.org/) with information related to the canidae family.
The database here used was converted to db format from a csv table used in our paper [4SpecID: Reference DNA Libraries Auditing and Annotation System for Forensic Applications](https://doi.org/10.3390/genes12010061) just for demonstration pruposes.

The datase includes two tables:
* species: with columns ['recordID', 'phylum_name', 'class_name', 'order_name', 'family_name',
       'genus_name', 'species_name', 'subspecies_name']
* bins: with columns ['recordID', 'bin_uri', 'nucleotides']

Both tables connect through a common columns, 'recordid'


The first thing we need is to create a pointer (squlite3 connection object) to our database and a function to get access to its entries, by taking advantadge of pandas. 

In [2]:
# connect database
db = sqlite3.connect('Canidae_COI.db')
def get_data(query):
    '''
    get data from the database
    '''
    return pd.read_sql(query, db)

## Tables in database

One database can contain one or more tables. The example here contain two (described above), but if we want to access data from a db without previous knowledge we can  access the number of tables and their names by using the command .execute as in the example below.

In [3]:
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tableNames = cursor.fetchall()
tableNames = [ tableNames[i][0] for i in range(len(tableNames)) ]
print(tableNames)

['species', 'bins']


To access specific tables and/or data inside tables several functions exist.

Let's start by investigating some SQL commands such as SELECT; WHERE; LIMIT,...


## Access database data


### SELECT
Command "SELECT" plus wildcard "*" to get the entire table


In [4]:
query = 'SELECT * FROM species;'
speciesDF = get_data(query)
speciesDF

,recordID,phylum_name,class_name,order_name,family_name,genus_name,species_name,subspecies_name
0,40935,Chordata,Mammalia,Carnivora,Canidae,Vulpes,Vulpes vulpes,None
1,546839,Chordata,Mammalia,Carnivora,Canidae,Urocyon,Urocyon cinereoargenteus,None
2,488348,Chordata,Mammalia,Carnivora,Canidae,Canis,Canis lupus,None
3,8458404,Chordata,Mammalia,Carnivora,Canidae,Vulpes,Vulpes chama,None
4,8458416,Chordata,Mammalia,Carnivora,Canidae,Vulpes,Vulpes vulpes,None
...,...,...,...,...,...,...,...,...
2004,1263606,Chordata,Mammalia,Carnivora,None,None,None,None
2005,1620215,Chordata,Mammalia,Carnivora,None,None,None,None
2006,1929546,Chordata,Mammalia,Carnivora,None,None,None,None
2007,6219159,Chordata,Mammalia,Carnivora,None,None,None,None


### LIMIT
A smaller version of the database could have been retrived by using LIMIT, to limit the number of output rows; and SELECT, to retrieve just a few variables

In [5]:
speciesSmallerDF = get_data('SELECT recordid, species_name, subspecies_name FROM species LIMIT 100;')
speciesSmallerDF

,recordID,species_name,subspecies_name
0,40935,Vulpes vulpes,None
1,546839,Urocyon cinereoargenteus,None
2,488348,Canis lupus,None
3,8458404,Vulpes chama,None
4,8458416,Vulpes vulpes,None
...,...,...,...
95,6170634,Canis lupus,None
96,6170662,Canis lupus,None
97,6170666,Canis lupus,None
98,6170677,Canis lupus,None


### ORDER BY

Data can be sorted according to some column of interest using "ORDER BY" followed by ASC or DESC for ascending or descending order.

In [6]:
speciesOrderSmallerDF = get_data('SELECT recordid, species_name, subspecies_name FROM species ORDER BY species_name DESC LIMIT 100;')
speciesOrderSmallerDF

,recordID,species_name,subspecies_name
0,6963578,Vulpes zerda,None
1,9630467,Vulpes zerda,None
2,6170884,Vulpes zerda,None
3,40935,Vulpes vulpes,None
4,8458416,Vulpes vulpes,None
...,...,...,...
95,9793074,Vulpes lagopus,None
96,9793073,Vulpes lagopus,None
97,9793076,Vulpes lagopus,None
98,9793053,Vulpes lagopus,None


### WHERE

Other keywords can be used to trim the recovered  database. For example "WHERE".
Let's get just data with subspecies assigned.

In [7]:
query = '''
SELECT recordid, species_name, subspecies_name
FROM species
WHERE ( subspecies_name != "None" ) 
ORDER BY subspecies_name
'''
speciesWithSubSpeciesDF = get_data(query)
speciesWithSubSpeciesDF

,recordID,species_name,subspecies_name
0,9630458,Canis lupus,Canis lupus chanco
1,1176624,Canis lupus,Canis lupus chanco
2,6959422,Canis lupus,Canis lupus chanco
3,3619539,Canis lupus,Canis lupus desertorum
4,3599939,Canis lupus,Canis lupus familiaris
...,...,...,...
293,6170867,Urocyon littoralis,Urocyon littoralis santarosae
294,6170864,Urocyon littoralis,Urocyon littoralis santarosae
295,6170868,Urocyon littoralis,Urocyon littoralis santarosae
296,6170863,Urocyon littoralis,Urocyon littoralis santarosae


## Logical operators
AND and OR operators can be used to filter even more

In [8]:
query = '''
SELECT recordid, species_name, subspecies_name
FROM species
WHERE ( subspecies_name != "None" AND species_name == "Canis lupus") 
ORDER BY subspecies_name
'''
speciesWithSubSpeciesDF = get_data(query)
speciesWithSubSpeciesDF

,recordID,species_name,subspecies_name
0,9630458,Canis lupus,Canis lupus chanco
1,1176624,Canis lupus,Canis lupus chanco
2,6959422,Canis lupus,Canis lupus chanco
3,3619539,Canis lupus,Canis lupus desertorum
4,3599939,Canis lupus,Canis lupus familiaris
...,...,...,...
134,6959685,Canis lupus,Canis lupus laniger
135,9630459,Canis lupus,Canis lupus laniger
136,10868366,Canis lupus,Canis lupus lupus
137,9630457,Canis lupus,Canis lupus lupus


## Statistics

### COUNT
Finally, to determine the number of entries that fulfilled a given set of criteria, we can get the table and determine its size, or, **more quickly**, use the COUNT

In [9]:
howManyCanisLupus = get_data('SELECT COUNT(*) FROM species WHERE species_name == "Canis lupus";')
howManyCanisLupus

,COUNT(*)
0,1544


### GROUP BY
Or we can use COUNT combined with GROUP BY to get counts by groups.

In [10]:
query = '''
SELECT species_name, COUNT(*) 
FROM species 
WHERE genus_name == "Canis"
GROUP BY species_name;
'''
howManyPerGenus = get_data(query)
howManyPerGenus

,species_name,COUNT(*)
0,None,1
1,Canis adustus,6
2,Canis anthus,2
3,Canis aureus,6
4,Canis familiaris,3
5,Canis latrans,15
6,Canis lupus,1544
7,Canis lycaon,1
8,Canis mesomelas,3


### MIN(), MAX(), and AVG()

Other mathematical functions can be used to retrieve results directly.
For example, the code below will get the average, minimum and maximum recordid among all Canis adustus (just demonstrations, the value is meaningless in this context).

In [11]:
get_data('SELECT AVG(recordid), MIN(recordid), MAX(recordid) FROM species WHERE species_name == "Canis adustus";')


,AVG(recordid),MIN(recordid),MAX(recordid)
0,5889701.5,5109911,9788642


## Connect/combine two tables

To combine multiple tables, we can SELECT columns by naming them table.columnName, e.g. species.recordid will correspond to the column recordid of table species, while bins.recordid will correspond to the recordid column in bins table.

To join two tables we can use JOIN: 
* FROM tableName1 INNER JOIN tableName2

and tell it to JOIN these two tables by column tableName1.x and tableName2.y we use 
* ON  tableName1.x = tableName2.y

### INNER JOIN and ON 

In [12]:
query = '''
SELECT species.recordid AS "RecordID", species.species_name AS "Species Name", bins.bin_uri AS "BIN"
FROM species INNER JOIN bins
ON species.recordid == bins.recordid
where species_name != "None"
ORDER BY bin_uri DESC
'''
allDataDF = get_data(query)
allDataDF

,RecordID,Species Name,BIN
0,9788642,Canis adustus,BOLD:AEE9377
1,11549569,Lycalopex gymnocercus,BOLD:AED1264
2,11549555,Lycalopex sechurae,BOLD:AED1264
3,11550007,Lycalopex gymnocercus,BOLD:AED1264
4,11549562,Lycalopex gymnocercus,BOLD:AED1264
...,...,...,...
1963,6170584,Canis lupus,BOLD:AAA1542
1964,6170606,Canis lupus,BOLD:AAA1542
1965,6170676,Canis lupus,BOLD:AAA1542
1966,5108268,Canis lupus,BOLD:AAA1542


Finally, we can combine previous commands and get the count of each BIN connected to a given species (e.g. "Vulpes vulpes")

In [13]:
query = '''
SELECT species.species_name AS Species, bins.bin_uri AS BIN, COUNT(*) AS Count
FROM species INNER JOIN bins
ON species.recordid == bins.recordid
WHERE species.genus_name == "Vulpes"
GROUP BY BIN
ORDER BY Species ASC, Count DESC
;
'''
get_data(query)

,Species,BIN,Count
0,Vulpes chama,BOLD:ADJ8649,1
1,Vulpes chama,BOLD:ADW1234,1
2,Vulpes corsac,BOLD:ADC8199,2
3,Vulpes ferrilata,BOLD:ADC8514,2
4,Vulpes lagopus,BOLD:AAC5231,63
5,Vulpes vulpes,BOLD:ADC5726,22
6,Vulpes vulpes,BOLD:AAC6751,12
7,Vulpes vulpes,BOLD:ADK6164,8
8,Vulpes zerda,BOLD:ADC5840,2
9,Vulpes zerda,BOLD:ACW0143,1
